# Exercise 18 Solution - Deep Energy Method for a Plate in Membrane Action
### Task
1. Implement the physics-informed loss for a two-dimensional plate in membrane action as `getLossTerms`
2. Solve the problem by executing the training loop
3. Introduce a more complex geometry through the indicator `alpha`. To this end, modify the function `getLossTerms`
4. Modify the indicator with the function `getIndicator` to provide a indicator that is one everywhere except at a centered circle with radius 0.2

### Learning goals
- Familiarize yourself with the deep energy method in a two-dimensional framework
- Understand the idea behind embedded domain methods (that discretize geometries through indicator functions instead of boundary conforming meshes/collocation points)

**import libraries & set seed**

In [ ]:
import numpy as np
import torch
from torch.autograd import grad
import time
import matplotlib.pyplot as plt

In [ ]:
torch.manual_seed(2)

## Utilities

**gradient computation with automatic differentiation**

In [ ]:
def getDerivative(y, x, n):
    if n == 0:
        return y
    else:
        dy_dx = grad(y, x, torch.ones(x.shape), create_graph=True, retain_graph=True)[0]
        return getDerivative(dy_dx, x, n - 1)

**neural network**

In [ ]:
class NN(torch.nn.Module):
    def __init__(
            self,
            inputDimension,
            hiddenDimensions,
            outputDimension,
            activationFunction=torch.nn.Tanh(),
    ):
        super().__init__()

        modules = []

        modules.append(torch.nn.Linear(inputDimension, hiddenDimensions[0]))
        modules.append(activationFunction)
        for i in range(len(hiddenDimensions) - 1):
            modules.append(
                torch.nn.Linear(hiddenDimensions[i], hiddenDimensions[i + 1])
            )
            modules.append(activationFunction)
        modules.append(torch.nn.Linear(hiddenDimensions[-1], outputDimension))

        self.model = torch.nn.Sequential(*modules)

    def forward(self, x):
        u = self.model(x)
        return u

**grid creation**

In [ ]:
def getGrid(Lx, Ly, Nx, Ny):
    x = torch.linspace(0, Lx, Nx)
    y = torch.linspace(0, Ly, Ny)
    x, y = torch.meshgrid(x, y, indexing="ij")
    x.requires_grad = True
    y.requires_grad = True
    input = torch.cat((x.reshape(-1, 1), y.reshape(-1, 1)), 1)
    return x, y, input

## PINN helper functions

**displacement computation**
$$\hat{\boldsymbol{u}}=(\hat{u},\hat{v})^\intercal=F_{NN}(x)$$

In [ ]:
def getDisplacements(model, x, strongEnforcementx, strongEnforcementy):
    u = model(x)
    ux = strongEnforcementx(u[:, 0], x[:, 0], x[:, 1]).unsqueeze(1)
    uy = strongEnforcementy(u[:, 1], x[:, 0], x[:, 1]).unsqueeze(1)
    u = torch.cat((ux, uy), 1)
    return u  # model(x)

**strain computation**
$$\boldsymbol{\varepsilon}=\begin{pmatrix}
\frac{\partial u}{\partial x}\\
\frac{\partial v}{\partial y}\\
\frac{1}{2}(\frac{\partial u}{\partial y} + \frac{\partial v}{\partial x})
\end{pmatrix}
$$

In [ ]:
def getStrains(u, x, y, Nx, Ny):
    strain = torch.zeros((3, Nx, Ny))
    strain[0] = getDerivative(u[:, 0].reshape(Nx, Ny), x, 1)
    strain[1] = getDerivative(u[:, 1].reshape(Nx, Ny), y, 1)
    strain[2] = 0.5 * (
            getDerivative(u[:, 0].reshape(Nx, Ny), y, 1)
            + getDerivative(u[:, 1].reshape(Nx, Ny), x, 1)
    )
    return strain

**loss term computation**

stress $$\boldsymbol{\sigma}=\boldsymbol{C}\boldsymbol{\varepsilon}$$

internal energy $$\Pi_i = \frac{1}{2} \int_{\Omega} \boldsymbol{\varepsilon}\cdot \boldsymbol{\sigma} d\Omega$$

external energy (assuming homogeneous Neumann boundary conditions) $$\Pi_e = -\int_{\Omega}\boldsymbol{p}\cdot\boldsymbol{u}d\Omega$$

In [ ]:
def getLossTerms(x, y, u, Nx, Ny, Lx, Ly, C, force, alpha):
    strain = getStrains(u, x, y, Nx, Ny)
    stress = torch.tensordot(C, strain, dims=1)

    # internal energy
    internalEnergy = 0.5 * torch.trapz(
        torch.trapz(alpha * torch.sum(stress * strain, 0), dx=Lx / (Nx - 1), dim=0),
        dx=Ly / (Ny - 1),
        dim=0,
    )

    # external energy
    externalEnergy = -torch.trapz(
        u[:, 0].reshape(Nx, Ny)[-1, :] * force, dx=Ly / (Ny - 1), dim=0
    )

    return internalEnergy, externalEnergy

## Problem setup

**physical parameters**

In [ ]:
# Problem data
Lx = 1
Ly = 1

E = 1.0
nu = 0.3
C = torch.zeros((3, 3))
C[0, 0] = 1
C[0, 1] = nu
C[1, 0] = nu
C[1, 1] = 1
C[2, 2] = 1 - nu
C *= E / (1 - nu ** 2)

# Neumann boundary condition on right edge
force = 1

# Dirichlet boundary conditions through strong enforcement
strongEnforcementx = lambda u, x, y: x * u
strongEnforcementy = lambda u, x, y: y * u


# Geometry through indicator
def getIndicator(x, y):
    alpha = torch.ones_like(x, dtype=torch.float)

    # # plate with a hole
    # radius = 0.2
    # index = (x-0.5)**2 + (y-0.5)**2 < radius**2
    # alpha[index] = 0
    return alpha

**hyperparameters**

currently Adam is selected as optimizer. By commenting the Adam block and uncommenting the LBFGS block, you can enable LBFGS as optimizer.

In [ ]:
Nx = 20  # number of collocation points in x
Ny = 20  # number of collocation points in y
hiddenDimensions = [20, 20, 20]
activationFunction = torch.nn.SiLU()

epochs = 300  # number of epochs
lr = 1e-2  # learning rate
selectOptimizer = "Adam"

**neural network & optimizer setup**

In [ ]:
model = NN(2, hiddenDimensions, 2, activationFunction)
optimizer = torch.optim.Adam(model.parameters(), lr)

**training grid**

In [ ]:
x, y, input = getGrid(Lx, Ly, Nx, Ny)
alpha = getIndicator(x, y)  # geometry through indicator

## Training
**cost function**
$$C=\Pi_i+\Pi_e$$

In [ ]:
internalEnergyHistory = np.zeros(epochs)
externalEnergyHistory = np.zeros(epochs)
costHistory = np.zeros(epochs)
start = time.perf_counter()
start0 = start
for epoch in range(epochs):
    optimizer.zero_grad()
    uPred = getDisplacements(model, input, strongEnforcementx, strongEnforcementy)

    lossTerms = getLossTerms(x, y, uPred, Nx, Ny, Lx, Ly, C, force, alpha)

    cost = lossTerms[0] + lossTerms[1]

    cost.backward()

    optimizer.step()

    internalEnergyHistory[epoch] = lossTerms[0].detach()
    externalEnergyHistory[epoch] = lossTerms[1].detach()
    costHistory[epoch] = lossTerms[0].detach() + lossTerms[1].detach()

    if epoch % 50 == 0:
        elapsedTime = (time.perf_counter() - start) / 50
        string = "Epoch: {}/{}\t\tDifferential equation cost = {:.2e}\t\tBoundary condition cost = {:.2e}\t\tTotal cost = {:.2e}\t\tElapsed time = {:2f}"
        print(
            string.format(
                epoch,
                epochs,
                internalEnergyHistory[epoch],
                externalEnergyHistory[epoch],
                costHistory[epoch],
                elapsedTime,
            )
        )
        start = time.perf_counter()
elapsedTime = time.perf_counter() - start0
string = "Total elapsed time: {:2f}\nAverage elapsed time per epoch: {:2f}"
print(string.format(elapsedTime, elapsedTime / epochs))

## Post-processing

**training history**

In [ ]:
fig, ax = plt.subplots()
ax.plot(costHistory, "k", label="cost")
ax.plot(internalEnergyHistory, "r:", label="internal energy")
ax.plot(externalEnergyHistory, "b:", label="external energy")
ax.grid()
ax.legend()
plt.show()

**displacement prediction**

In [ ]:
Nx_ = 100
Ny_ = 100
x_, y_, input_ = getGrid(Lx, Ly, Nx_, Ny_)

upred_ = getDisplacements(model, input_, strongEnforcementx, strongEnforcementy)
strain_ = getStrains(upred_, x_, y_, Nx_, Ny_)

alpha_ = getIndicator(x_, y_)
maskedu1pred_ = np.ma.masked_array(upred_[:, 0].detach(), ~(alpha_.detach() > 0)).reshape(Nx_, Ny_)
maskedu2pred_ = np.ma.masked_array(upred_[:, 1].detach(), ~(alpha_.detach() > 0)).reshape(Nx_, Ny_)

fig, ax = plt.subplots()
cp = ax.pcolormesh(
    x_.detach(), y_.detach(), maskedu1pred_, cmap=plt.cm.jet, shading='auto'
)
fig.colorbar(cp)
ax.plot(x.detach(), y.detach(), "k.")
plt.gca().set_aspect("equal", adjustable="box")
ax.set_title("$u$")
plt.show()

fig, ax = plt.subplots()
cp = ax.pcolormesh(
    x_.detach(), y_.detach(), maskedu2pred_, cmap=plt.cm.jet, shading='auto'
)
fig.colorbar(cp)
ax.plot(x.detach(), y.detach(), "k.")
plt.gca().set_aspect("equal", adjustable="box")
ax.set_title("$v$")
plt.show()